In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from transformers import TFAutoModel, AutoTokenizer

In [ ]:
model = TFAutoModel. from_pretrained('bert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
from datasets import load_dataset

In [ ]:
# Load the local CSV file
dataset = load_dataset('csv', data_files='modified_train.csv')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['comment', 'label'],
        num_rows: 213747
    })
})

In [ ]:
def tokenize(batch):
    return tokenizer(
        batch['comment'],
        padding='max_length',   # Pad to max_length
        truncation=True,        # Truncate if longer than max_length
        max_length=20          # Specify the desired length for padding
    )

In [ ]:
dataset_encoded = dataset.map(tokenize, batched=True, batch_size=32)

Map:   0%|          | 0/213747 [00:00<?, ? examples/s]

In [ ]:
dataset_encoded

DatasetDict({
    train: Dataset({
        features: ['comment', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 213747
    })
})

In [ ]:
import tensorflow as tf

In [ ]:
# setting 'input_ids', 'attention_mask', 'token_type_ids', and 'label'
# to the tensorflow format. Now if you access this dataset you will get these
# columns in `tf.Tensor` format

dataset_encoded.set_format('tf',
                            columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

# setting BATCH_SIZE to 64.
BATCH_SIZE = 32

def order(inp):
    '''
    This function will group all the inputs of BERT
    into a single dictionary and then output it with
    labels.
    '''
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]
# converting train split of `emotions_encoded` to tensorflow format

train_dataset = tf.data.Dataset.from_tensor_slices(dataset_encoded['train'][:])
# set batch_size and shuffle
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
# map the `order` function
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
history = classifier.fit(
    train_dataset,
    epochs=3
)

Epoch 1/3
6680/6680 [==============================] - 1024s 152ms/step - loss: 0.6032 - accuracy: 0.7419
Epoch 2/3
6680/6680 [==============================] - 1017s 152ms/step - loss: 0.5374 - accuracy: 0.7716
Epoch 3/3
6680/6680 [==============================] - 1016s 152ms/step - loss: 0.4746 - accuracy: 0.8004


In [ ]:
import pandas as pd
from datasets import Dataset
import tensorflow as tf

# 1. Load the test.csv file into a pandas DataFrame
test_df = pd.read_csv('test.csv')

# Assuming 'id' and 'comment' columns exist in your test.csv file
ids = test_df['id'].tolist()  # Extract 'id' column

# 2. Tokenize the test data
def tokenize_test(batch):
    return tokenizer(
        batch['comment'],        # Use the 'comment' column for tokenization
        padding='max_length',    # Pad to the maximum length
        truncation=True,         # Truncate if longer than max_length
        max_length=10            # Specify the maximum length for padding
    )

# Convert the pandas DataFrame to a Hugging Face Dataset
test_dataset = Dataset.from_pandas(test_df)

# Tokenize the dataset
test_encoded = test_dataset.map(tokenize_test, batched=True, batch_size=32)

# 3. Set the format to TensorFlow tensors
test_encoded.set_format('tf', columns=['input_ids', 'attention_mask', 'token_type_ids'])

# Convert the encoded test dataset to a TensorFlow Dataset
test_dataset_tf = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': test_encoded['input_ids'],
        'attention_mask': test_encoded['attention_mask'],
        'token_type_ids': test_encoded['token_type_ids']
    }
))

# Set batch size (you can modify this as needed)
BATCH_SIZE = 32
test_dataset_tf = test_dataset_tf.batch(BATCH_SIZE)

# 4. Make predictions using the trained classifier model
predictions = classifier.predict(test_dataset_tf)

# 5. Convert logits to class labels
predicted_class_indices = tf.argmax(predictions, axis=-1).numpy()

# Define your label mapping (adjust according to your use case)
label_mapping = {0: 'doubt', 1: 'feedback', 2: 'irrelevant'}
predicted_labels = [label_mapping[idx] for idx in predicted_class_indices]

# 6. Create a DataFrame with 'id' and 'predictions'
output_df = pd.DataFrame({
    'id': ids,
    'label': predicted_labels
})

# Save the predictions to predictions.csv with only 'id' and 'label' columns
output_df.to_csv('predictions_llm.csv', index=False)

print("Predictions saved to predictions.csv with 'id' and 'label'.")


Map:   0%|          | 0/91607 [00:00<?, ? examples/s]

2863/2863 [==============================] - 104s 36ms/step
Predictions saved to predictions.csv with 'id' and 'label'.


In [ ]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model, label):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(label, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [ ]:
classifier = BERTForClassification(model, label=3)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)